In [1]:
import mwpf
mwpf.Visualizer.embed()

# Parity Matrix Tutorial

We define Parity Matrix in our paper, which represents the complete parity check constraints over some vertices $V_M$ and edges $E_M$.
The solution space of parity factor is written as $\vec{x} = \{x_e | e \in E_M\} \subseteq \mathbb{Z}_2^{|E_M|}$ and each edge has a binary variable $x_e \in \{ 0, 1 \}$ representing the edge in the parity factor solution or not.
There is one parity constraint associated with each vertex, namely $\sum_{e \in E_M | v \in e} x_e = \begin{cases}1, & v\in D\\0, & v \notin D\end{cases}\mod 2$ where $D$ is the set of defect vertices that indicates an odd number of errors (edges) have flipped it.

In order to provide the functionality required for this project, the Rust implementation of the Parity Matrix is a nested Generic implementation.
- `BasicMatrix`: the basic parity matrix
- `TightMatrix = Tight<BasicMatrix>`: adding the ability to hide loose edges and only show tight edges in the matrix
- `TailMatrix = Tail<TightMatrix>`: adding the ability to reorder columns of the parity matrix
- `EchelonMatrix = Echelon<TailMatrix>`: adding the ability to calculate the reduced row echelon form

Each of them can be converted to another (a new object) via `get_base(self)` and `ClassName(base)` functions.
For example, given a basic matrix `basic_matrix`, we can construct a new tight matrix by `tight_matrix = TightMatrix(basic_matrix)`.
We can go back to the basic matrix with `basic_matrix_2 = tight_matrix.get_base()`, which returns a new matrix.
One can also convert the class explicitly through multiple levels, for example, `BasicMatrix(echelon_matrix)` and `EchelonMatrix(basic_matrix)` are all valid constructions.

In the following sections, we'll go through the capabilities of individual matrices.

## BasicMatrix

We first look at a very simple parity matrix implementation that offers full customization of columns and rows.

The matrix has the following APIs:
- `add_variable(self, edge_index: int) -> int | None`: return the variable index (column index) in the matrix if added; otherwise if the edge already exists in the matrix, return `None`. User can use `edge_to_var_index` below to get the variable index instead.
- `add_constraint(self, vertex_index: int, incident_edges: list[int], parity: bool) -> list[int] | None`: add a row in the parity matrix with the vertex. If the vertex constraint has already been added, return `None` (to avoid duplicate rows). User don't have to add all the variables before they add a constraint. All the missing variables (columns) will be automaticall added. It will return a list of variable indices of newly added edges.
- `get_lhs(self, row: int, var_index: int) -> bool`: get left-hand side value
- `get_rhs(self, row: int) -> bool`: get right-hand side value
- `var_to_edge_index(self, var_index: int) -> int`
- `edge_to_var_index(self, edge_index: int) -> int`
- `exists_edge(self, edge_index: int) -> bool`
- `get_vertices(self) -> set[int]`

The following APIs are for matrix view. For `BasicMatrix`, the visible matrix when you call `matrix.show()` or `print(matrix)` is exactly the original parity matrix. For more complicated matricies, the columns or the rows might change.
- `columns: int`: property, the number of columns in the visual view; For `BasicMatrix`, column index is the variable index; for more complicated matrices that supports hiding some variables (`TightMatrix`) or reorder the variables (`TailMatrix`), the visual column index does not immediately corresponds to variable index, and the following functions provide the translation.
- `column_to_var_index(self, column: int) -> int`
- `column_to_edge_index(self, column: int) -> int`
- `get_view_edges(self) -> list[int]`: return the edge indices of the visble columns
- `var_to_column_index(self, var_index: int) -> int | None`: return the column index if the variable is visible, otherwise return `None`
- `edge_to_column_index(self, edge_index: int) -> int | None`: return the column index if the edge is visible, otherwise return `None`
- `rows: int`: property, the number of rows in the visual view; For `BasicMatrix`, the number of vertices is the number of rows; for more complicated matrices that supports hiding zero rows (`EchelonMatrix`), the number of rows may be less than the number of vertices.


In [2]:
matrix = mwpf.BasicMatrix()
print(matrix)  # you can print in the text form
matrix.show()  # or with a better HTML form with colors and highlights

┌┬───┐
┊┊ = ┊
╞╪═══╡
└┴───┘



Then we can add some variables to the matrix. Each variable corresponds to a column in the parity matrix.

In [3]:
matrix.add_variable(edge_index=1)
matrix.add_variable(12)
matrix.add_variable(4)
matrix.add_variable(345)
matrix.show()

We can then add some constraint. Each constraint is a row in the parity matrix, corresponding to a vertex?

In [4]:
matrix.add_constraint(0, [1, 4, 12], True)
matrix.add_constraint(3, [4, 345], False)
matrix.add_constraint(4, [1, 345], True)
matrix.show()

We can do row operations on this matrix: `xor_row` and `swap_row`. The arguments are the indices of the rows.

In [5]:
matrix.xor_row(target=2, source=0)
matrix.show()

We can do a sequence of operations, which gives us a reduced row Echelon form matrix.

In [6]:
matrix.swap_row(1, 2)
matrix.xor_row(0, 1)
matrix.xor_row(1, 2)
matrix.show()

## TightMatrix

In the above `BasicMatrix`, we consider all edges as columns.
However, often times we add an edge but will not select it (force $x_e = 0$).
For example, in MWPF decoder, we use the complementary slackness theorem to infer which edges can be selected in the optimal MWPF solution.
The theorem says $$\sum_{S\in\mathcal{O}|e \in \delta(S)} y_S < w_e \Longrightarrow x_e = 0$$, i.e., only tight edges ($\sum_{S\in\mathcal{O}|e \in \delta(S)} y_S = w_e$) can be possibly chosen ($x_e \in \{0,1\}$), loose edges ($\sum_{S\in\mathcal{O}|e \in \delta(S)} y_S < w_e$) will never be choosen ($x_e = 0$).

We want the matrix to only contain columns that corresponds to tight edges, while still maintains those loose edges just in case they become tight in the future.
We wrap it as a `TightMatrix` class, which offers the following new functions:
- `update_edge_tightness(self, edge_index: int, is_tight: bool)`
- `is_tight(edge_index: int) -> bool`
- `add_variable_with_tightness(self, edge_index: int, is_tight: bool)`
- `add_tight_variable(self, edge_index: int)`

By using `TightMatrix`, we assume edges by default are loose.
Thus, newly added variables do not automatically show up.
You need to either use `update_edge_tightness` to modify the tightness of the edge, or use `add_tight_variable` to directly add the edge as tight edge.

In [7]:
matrix = mwpf.TightMatrix()
matrix.add_constraint(vertex_index=0, incident_edges=[1, 4, 6], parity=True)
matrix.add_constraint(1, [4, 9], parity=False)
matrix.add_constraint(2, [1, 9], parity=True)
matrix.show()

Since edges are loose by default, you will not see any variable columns in the parity matrix. To show some of the variables, we just need to set some of the edges to tight.

In [8]:
matrix.update_edge_tightness(4, True)
matrix.update_edge_tightness(9, True)
matrix.show()

Of course, you can set an edge back to loose state.

In [9]:
matrix.update_edge_tightness(9, False)
matrix.show()

## TailMatrix

Tail matrix allows us to put some of the edges towards the end of the matrix.
This is useful in the MWPF algorithm where we want the hair $\delta(S)$ of a non-zero dual variable $y_S > 0$ to be mostly to the right side so that it is isolated from the other dual variables.
Built upon `TightMatrix`, the `TailMatrix` also default the edges to be loose, and thus we will not see anything below.

In [10]:
matrix = mwpf.TailMatrix()
matrix.add_constraint(vertex_index=0, incident_edges=[1, 4, 6], parity=True)
matrix.add_constraint(1, [4, 9], parity=False)
matrix.add_constraint(2, [1, 9], parity=True)
matrix.show()

Now we set all the edges to be tight.

In [11]:
for edge_index in [1, 4, 6, 9]:
    matrix.update_edge_tightness(edge_index, True)
matrix.show()

We can set edges $\{1,6\}$ to be the tail of the matrix (towards the right side).
Note that we cannot specify the order within the tail edges.
The edges will be automatically ordered according to their variable index.
(note: it is possible to force the order of the edges in Rust, by using some type like `Tail<CustomReorder<...>>` where `CustomReorder` will change the view of the matrix. However, our MWPF algorithm doesn't care much about the order so we do not use it for performance considerations)

In [12]:
matrix.set_tail_edges({6, 1})
assert matrix.get_tail_edges() == {1, 6}
matrix.show()

To revert the process, just set the tail edges to empty set.

In [13]:
matrix.set_tail_edges({})
matrix.show()

## EchelonMatrix

The reduced row Echelon form is a powerful tool to reason about the solution space among the tight edges.
For example, if we have a graph that looks like this:

In [14]:
import math
vertex_num = 3
positions = [mwpf.VisualizePosition(i, j, 0) for (i, j) in [(0, 1-math.sqrt(3)), (-1, 1), (1, 1)]]
weighted_edges = [
    mwpf.HyperEdge({0, 2}, 1),
    mwpf.HyperEdge({0, 1}, 1),
    mwpf.HyperEdge({1, 2}, 1),
    mwpf.HyperEdge({0, 1, 2}, 1),
]
initializer = mwpf.SolverInitializer(vertex_num, weighted_edges)
solver = mwpf.Solver(initializer)
syndrome = mwpf.SyndromePattern({1})
solver.load_syndrome(syndrome)
visualizer = mwpf.Visualizer(positions=positions)
visualizer.snapshot("syndrome", solver)
# add dual variables
S0 = solver.get_node(0)  # S0 = ({1}, {})
S1 = solver.create_node({0,1,2}, {1,2,3})
solver.grow(mwpf.Rational(1))
visualizer.snapshot("grow 1", solver)
visualizer.show()

The basic parity matrix of this state is:

In [15]:
matrix = mwpf.TightMatrix()
for edge_index in [0, 1, 2, 3]:
    matrix.add_tight_variable(edge_index)  # all edges are tight
matrix.add_constraint(vertex_index=0, incident_edges={1, 3, 0}, parity=False)
matrix.add_constraint(1, {1, 3, 2}, parity=True)
matrix.add_constraint(2, {2, 3, 0}, parity=False)
matrix.show()

The basic parity matrix does not give too much information about how to choose the edges: the choice of one edge will affect the possible choice of others.
The reduced row Echelon form will help us explore the structure within this cluster.

In [16]:
matrix = mwpf.EchelonMatrix(matrix)
matrix.show()

Given this reduced row Echelon form matrix, we are essentially transforming the decoding hypergraph into another form where the edges are still the same but the vertices are XOR'd with each other.
The decoding hypergraph now looks like this:

In [17]:
weighted_edges = [
    mwpf.HyperEdge({0}, 1),
    mwpf.HyperEdge({1}, 1),
    mwpf.HyperEdge({0, 1}, 1),
    mwpf.HyperEdge({2}, 1),
]
initializer = mwpf.SolverInitializer(vertex_num, weighted_edges)
solver = mwpf.Solver(initializer)
syndrome = mwpf.SyndromePattern({0, 2})  # Attention! the vertices have a different syndrome, given by the RHS of the parity matrix above
solver.load_syndrome(syndrome)
visualizer = mwpf.Visualizer(positions=positions)
visualizer.snapshot("syndrome", solver)
# add dual variables; now the dual variables also changes.
solver.stop_all()
S0 = solver.create_node_hair_unchecked({1, 2, 3})  # original S0, hair = {1,2,3}
S1 = solver.create_node_hair_unchecked({0})  # original S1, hair = {0}
solver.grow(mwpf.Rational(1))
visualizer.snapshot("grow 1", solver)
visualizer.show()

Now it becomes clear in the new decoding hypergraph that the blue dual variable (corresponds to the original S0) has a strange configuration: it could have grow on just one edge $\{3\}$ instead of on three edges $\{1,2,3\}$.
This helps us make a decision that we should shrink $y_{S_0}$ and then increase the dual variable $S2 = (\{v'_{2}\}, \varnothing)$.

In [18]:
S2 = solver.create_node({2})
solver.set_grow_rate(S0, mwpf.Rational(-1))
solver.set_grow_rate(S1, mwpf.Rational(0))
solver.grow(mwpf.Rational(1))
visualizer.snapshot("shrink 1 + grow 1", solver)
visualizer.show()

And then it becomes clear that we should choose edges $\{0, 3\}$ as the MWPF solution, whose primal summation is $\sum_{e\in E} x_e = 2$ and dual summation is also $\sum_{S\in \mathcal{O}} y_S = 2$.